In [351]:
import pandas as pd 
import numpy as np
df = pd.read_parquet('../scripts/df_merged.parquet', engine='pyarrow')

In [352]:
df.shape

(1831533, 10)

In [3]:
df.columns

Index(['objectId', 'candid', 'jd', 'mjd', 'fid', 'dc_flux', 'dc_sigflux',
       'nr_flux', 'nr_sigflux', 'source'],
      dtype='object')

In [4]:
df[df['objectId'] == 'ZTF17aaaeclk'][['mjd', 'fid', 'dc_flux', 'dc_sigflux', 'nr_flux', 'nr_sigflux', 'source']].head(1)

,mjd,fid,dc_flux,dc_sigflux,nr_flux,nr_sigflux,source
0,59080,1,0.000286,0.000027,0.000286,0.000004,1


In [5]:
unique_ids = df['objectId'].unique().tolist()

In [6]:
for i in unique_ids[0:1]:
    print(i)
    #print(df[df['objectId'] == i][['mjd', 'fid', 'dc_flux', 'dc_sigflux', 'nr_flux', 'nr_sigflux', 'source']])

ZTF17aaaeclk


In [7]:
condition = df['source'] == 0
df['dc_sigflux_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

In [8]:
df[['source','nr_sigflux_weight']]

,source,nr_sigflux_weight
0,1,6.391653e+10
1,1,7.280090e+09
2,0,0.000000e+00
3,0,0.000000e+00
4,0,0.000000e+00
...,...,...
1831528,0,0.000000e+00
1831529,0,0.000000e+00
1831530,0,0.000000e+00
1831531,1,2.882772e+09


In [218]:
T1 = df[df['objectId'] == 'ZTF17aaaeclk'][['mjd', 'fid',
                                           'dc_flux', 'dc_sigflux_weight',
                                           'nr_flux', 'nr_sigflux_weight',
                                           'source']]

In [219]:
m = 1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l = 1
f_r = [None,None]#[0] * m
w_r = [None,None]#[0] * m
h_r = [None,None]#[0] * m
alpha_r = [None,None]#[0] * m
d_r = [None,None]#[0] * m

In [220]:
f_r[1] = T1[T1['fid'] == 2]['dc_flux']
w_r[1] = T1[T1['fid'] == 2]['dc_sigflux_weight']
h_r[1] = T1[T1['fid'] == 2]['dc_flux'].loc[0:m]

In [221]:
h_r[1]

1    0.000636
Name: dc_flux, dtype: float64

In [222]:
s_n =0
s_d=0
for i in range(len(w_r[1])):
    s_n += (h_r[1].iloc[0] * f_r[1].iloc[i:i+m] * w_r[1].iloc[i:i+m]).item()
    s_d += (h_r[1].iloc[0]**2 * w_r[1].iloc[i:i+m]).item()
alpha_r[1]  = s_n / s_d
alpha_r[1]

1.0679293960612513

In [223]:
s =0
for i in range(len(w_r[1])):
    s += ((f_r[1].iloc[i:i+m] - alpha_r[1]* h_r[1].iloc[0])**2 * w_r[1].iloc[i:i+m] ).item()

d_r[1]  = s
d_r[1]

247.02033420794174

In [224]:
f_g = [None,None]#[0] * m
w_g = [None,None]#[0] * m
h_g = [None,None]#[0] * m
alpha_g = [None,None]#[0] * m
d_g = [None,None]#[0] * m

In [228]:
f_g[1] = T1[T1['fid'] == 1]['dc_flux']
w_g[1] = T1[T1['fid'] == 1]['dc_sigflux_weight']
h_g[1] = T1[T1['fid'] == 1]['dc_flux'].loc[0:1]
h_g

[None,
 0    0.000286
 Name: dc_flux, dtype: float64]

In [226]:
s_n =0
s_d=0
for i in range(len(w_g[1])):
    s_n += (h_g[1].iloc[0] * f_g[1].iloc[i:i+m] * w_g[1].iloc[i:i+m]).item()
    s_d += (h_g[1].iloc[0]**2 * w_g[1].iloc[i:i+m]).item()
alpha_g[1]  = s_n / s_d
alpha_g[1]

0.9598034477576466

In [227]:
s =0
for i in range(len(w_g[1])):
    s += ((f_g[1].iloc[i:i+m] - alpha_g[1]* h_g[1].iloc[0])**2 * w_g[1].iloc[i:i+m]).item()

d_g[1]  = s
d_g[1]

1433.2969709333252

In [169]:
d  = d_r[1] + d_g[1]
d

1680.3173051412668

In [170]:
d <= factor

False

In [171]:
factor

5.242640687119286

In [172]:
T1

,mjd,fid,dc_flux,dc_sigflux_weight,nr_flux,nr_sigflux_weight,source
0,59080,1,0.000286,1.353949e+09,0.000286,6.391653e+10,1
1,59080,2,0.000636,2.615609e+08,0.000636,7.280090e+09,1
2,59081,2,0.000679,0.000000e+00,0.000636,0.000000e+00,0
3,59081,1,0.000275,0.000000e+00,0.000286,0.000000e+00,0
4,59082,2,0.000679,0.000000e+00,0.000636,0.000000e+00,0
...,...,...,...,...,...,...,...
57,59108,2,0.000679,0.000000e+00,0.000636,0.000000e+00,0
58,59109,2,0.000679,0.000000e+00,0.000636,0.000000e+00,0
59,59109,1,0.000275,0.000000e+00,0.000286,0.000000e+00,0
60,59110,2,0.000891,2.117272e+09,0.000636,7.280090e+09,1


In [323]:
df[['fid','dc_flux']]

,fid,dc_flux
0,1,0.000286
1,2,0.000636
2,2,0.000679
3,1,0.000275
4,2,0.000679
...,...,...
1831528,2,0.001819
1831529,2,0.001819
1831530,1,0.001993
1831531,1,0.001581


In [350]:
df[['fid', 'dc_flux']].iloc[:2*m]

,fid,dc_flux
0,1,0.000286
1,2,0.000636


In [341]:
 df

,objectId,candid,jd,mjd,fid,dc_flux,dc_sigflux,nr_flux,nr_sigflux,source,dc_sigflux_weight,nr_sigflux_weight
0,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59080,1,0.000286,0.000027,0.000286,3.955427e-06,1,1.353949e+09,6.391653e+10
1,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59080,2,0.000636,0.000062,0.000636,1.172011e-05,1,2.615609e+08,7.280090e+09
2,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.000000e+00,0.000000e+00
3,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,1,0.000275,0.000119,0.000286,5.598796e-20,0,0.000000e+00,0.000000e+00
4,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59082,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
1831528,ZTF18abgjszd,1356153201015010008,2.459111e+06,59108,2,0.001819,0.000150,0.001902,0.000000e+00,0,0.000000e+00,0.000000e+00
1831529,ZTF18abgjszd,1356153201015010008,2.459111e+06,59109,2,0.001819,0.000150,0.001902,0.000000e+00,0,0.000000e+00,0.000000e+00
1831530,ZTF18abgjszd,1356153201015010008,2.459111e+06,59109,1,0.001993,0.000331,0.001838,4.540264e-10,0,0.000000e+00,0.000000e+00
1831531,ZTF18abgjszd,1356153201015010008,2.459111e+06,59110,1,0.001581,0.000028,0.001838,1.862494e-05,1,1.240177e+09,2.882772e+09


In [349]:
m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?
print(Q)


count = 0
for k in unique_ids[0:1]:
    #print(k)
    T = df[df['objectId'] == k][['mjd', 'fid',
                                           'dc_flux', 'dc_sigflux_weight',
                                           'nr_flux', 'nr_sigflux_weight',
                                           'source']]
    for l in [0]:
        # we don't use it rn 
        for j in range(1,m+1): 
            d=0
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                #print(filt)
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values
                print(Q)                
                #print(f[j::j])                   
                #print(Q[:j] * f[j::j])  
                
                s_n = (Q[:j] * f[:] * w[:])
                s_d = (Q[:j]**2 * w[:])

                alpha = s_n / s_d
                #print("alpha", alpha)
                
                d += np.sum(((f[::j] - alpha * Q[:j])**2) * w[::j])

                #print()
                #print(d)

                #break
            
    if d <= factor: 
        #print(d)
        count +=1
        print(count, k)
        
print(count)
   
        

   fid   dc_flux
0    1  0.000286
1    2  0.000636
   fid   dc_flux
0    1  0.000286
1    2  0.000636


ValueError: Unable to coerce to Series, length must be 2: given 31

In [310]:
Q = df['dc_flux'].iloc[0:m].values
Q

array([0.0002863])

In [313]:
f*Q*w

array([1272.17541827, 1293.89931474, 1299.801054  ,  675.11231853,
       1318.1653299 ,  740.9077833 ,   27.64713756,  735.92550232,
        729.71713492,  626.58400586,  692.22452168,  890.3436218 ,
          0.        ,  432.88527886,  501.195918  ,  623.78221033,
        610.87485025,  664.37520522,    0.        ,    0.        ,
          0.        ,    0.        ,  450.71877465,    0.        ,
        303.20291098,    0.        ,    0.        ,  383.97890241,
          0.        ,  751.07178494, 1426.36620699])

In [254]:
for j in range(1,m+1): 
    print(j)

1
2


In [255]:
T[mask]['dc_flux'].iloc[0:2]

1    0.000636
2    0.000679
Name: dc_flux, dtype: float64

In [260]:
Q.iloc[0]

1    0.000636
2    0.000679
Name: dc_flux, dtype: float64

In [192]:
df[df['objectId'] == unique_ids[0]].head(6)

,objectId,candid,jd,mjd,fid,dc_flux,dc_sigflux,nr_flux,nr_sigflux,source,dc_sigflux_weight,nr_sigflux_weight
0,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59080,1,0.000286,0.000027,0.000286,3.955427e-06,1,1.353949e+09,6.391653e+10
1,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59080,2,0.000636,0.000062,0.000636,1.172011e-05,1,2.615609e+08,7.280090e+09
2,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.000000e+00,0.000000e+00
3,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,1,0.000275,0.000119,0.000286,5.598796e-20,0,0.000000e+00,0.000000e+00
4,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59082,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.000000e+00,0.000000e+00
5,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59082,1,0.000286,0.000066,0.000286,3.955427e-06,1,2.325079e+08,6.391653e+10


In [302]:

import timeit

def approach1():
    condition = df['source'] == 0
    df['dc_sigflux_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
    df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

def approach2():
    df['dc_sigflux_weight'] = 1 / (df['dc_sigflux'] ** 2)
    df['nr_sigflux_weight'] = 1 / (df['nr_sigflux'] ** 2)
    df['dc_sigflux_weight'] = df['dc_sigflux_weight'] * (df['source'] != 0)
    df['nr_sigflux_weight'] = df['nr_sigflux_weight'] * (df['source'] != 0)

time_approach2 = timeit.timeit(approach2, number=10)
time_approach1 = timeit.timeit(approach1, number=10)

print("1 execution time:", time_approach1)
print("2 execution time:", time_approach2)


1 execution time: 0.2659939930017572
2 execution time: 0.47643366300326306
